In [104]:
# Import dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd
import time

In [107]:
# Initialize splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [108]:
'''Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
Assign the text to variables that you can reference later.'''

url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

# Retrieve page & parse with BeautifulSoup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [109]:
news_items = soup.find('ul', class_='item_list')
news_title = news_items.find('h3').text
news_para = news_items.find('div', class_='article_teaser_body').text
print(news_title)
print(news_para)

NASA's Curiosity Mars Rover Snaps Its Highest-Resolution Panorama Yet
To go along with the stunning 1.8-billion-pixel image, a new video offers a sweeping view of the Red Planet.


## JPL Mars Space Images - Featured Image

In [53]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

# Use splinter to navigate the site and find the image url for the current Featured Mars Image and 
# assign the url string to a variable called featured_image_url.
s = soup.find('article', class_='carousel_item')['style']
url = s.split("'")[1]
featured_image_url = 'https://www.jpl.nasa.gov' + url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23341-1920x1200.jpg


## Mars Weather (Twitter)

In [85]:
'''
Visit the Mars Weather twitter account and scrape the latest Mars weather tweet from the page. 
Save the tweet text for the weather report as a variable called mars_weather.
'''

url = 'https://twitter.com/marswxreport?lang=en'

html_text = requests.get(url).text
soup = BeautifulSoup(html_text, 'html.parser')

In [102]:
mars_weather = soup.find_all('p', class_='tweet-text')[0].text
mars_weather = mars_weather.split('pic.twitter.com')
mars_weather = mars_weather[0]

In [103]:
print(mars_weather)

InSight sol 451 (2020-03-03) low -93.6ºC (-136.4ºF) high -10.3ºC (13.4ºF)
winds from the SSE at 6.3 m/s (14.2 mph) gusting to 19.9 m/s (44.5 mph)
pressure at 6.30 hPa


## Mars Facts

In [74]:
# Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
r = requests.get("https://space-facts.com/mars/")
soup = BeautifulSoup(r.content,'lxml')
table = soup.find('table', id='tablepress-p-mars-no-2')
df = pd.read_html(str(table))
df = df[0]

df.columns = ['description', 'value']
df.set_index(keys=['description'], inplace=True)

# Use Pandas to convert the data to a HTML table string.
html = df.to_html()
html_table = html.replace('\n', '')
html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>value</th>    </tr>    <tr>      <th>description</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

## Mars Hemispheres

In [75]:
r = requests.get("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")
soup = BeautifulSoup(r.content,'lxml')

In [76]:
links = soup.find_all('a', class_='itemLink')

In [77]:
hemisphere_image_urls = []

for link in links:
    hemisphere = {}
    r = requests.get("https://astrogeology.usgs.gov/" + link['href'])
    soup = BeautifulSoup(r.content,'lxml')
    l = soup.find('li')
    link = l.contents[0]
    img_url = link['href']
    title = soup.title.text
    title = title.split("|")[0]
    hemisphere["title"] = title
    hemisphere["img_url"] = img_url
    hemisphere_image_urls.append(hemisphere)
    time.sleep(1)

In [78]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]